In [7]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [8]:
# Load Data
model = load_model("model1000000w.tf")
    
with open('MetaData/context_words.pickle', 'rb') as file:
    context_words = pickle.load(file)
with open('MetaData/unique_tokens.pickle', 'rb') as file:
    unique_tokens = pickle.load(file)
with open('MetaData/unique_token_index.pickle', 'rb') as file:
    unique_token_index = pickle.load(file)

In [9]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, context_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
                             
    predictions = model.predict(X,verbose=0)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [10]:
possible = predict_next_word("Once Upon a time there was a man who was",5)
print([unique_tokens[idx] for idx in possible])

['run', 'near', 'still', 'river', 'coming']


In [11]:
tokenizer = RegexpTokenizer(r"\w+")
def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+context_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [12]:
generate_text("Once Upon a time there was a man who was",100,1)

'Once Upon a time there was a man who was coming there with a long plain and often medicine get all winter and they had no hope for anything we ll reached book his strength and he was sure that he ll not live said tilly as he got out i am going to my table i ve been reached without a sheep in texas the five and four others for its red smile nor shoulders the ear they familiar on twenty hours of the desire going to enjoy the shepherd got up with her ears are present and be this fact yet i never hear mean you hear polly'